<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Finding-Shortest-Paths" data-toc-modified-id="Finding-Shortest-Paths-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Finding Shortest Paths</a></span></li><li><span><a href="#Limit-data-to-make-some-things-faster" data-toc-modified-id="Limit-data-to-make-some-things-faster-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Limit data to make some things faster</a></span><ul class="toc-item"><li><span><a href="#Degree-Centrality" data-toc-modified-id="Degree-Centrality-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Degree Centrality</a></span></li><li><span><a href="#Closeness-Centrality" data-toc-modified-id="Closeness-Centrality-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Closeness Centrality</a></span></li><li><span><a href="#Eigenvector-Centrality" data-toc-modified-id="Eigenvector-Centrality-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Eigenvector Centrality</a></span></li><li><span><a href="#Betweenness-centrality" data-toc-modified-id="Betweenness-centrality-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Betweenness centrality</a></span></li></ul></li></ul></div>

In [ ]:
import pandas as pd
import networkx as nx
from itertools import combinations

**Download Data from [Kaggle](https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset)**

Extract it into `/baconapi/data`

## Finding Shortest Paths

In [ ]:
movies = pd.read_csv("baconapi/data/movies.csv")
movies.head(3)

In [ ]:
G = nx.Graph()

In [ ]:
G.add_edges_from()

In [ ]:
nx.draw_networkx(G)

In [ ]:
movies = movies[movies["actors"].apply(lambda x: type(x)==str)]

In [ ]:
G = nx.Graph()
for i, row in movies.iterrows():
    pairs = list(combinations(map(lambda x:x.strip(),row["actors"].split(",")),2))
    G.add_edges_from(pairs)
    for pair in pairs:
        G.edges[pair]["movie"] = G.edges[pair].get("movie",[]) + [row["original_title"]]

In [ ]:
actors = G.nodes()

In [ ]:
len(actors)

In [ ]:
connections = G.edges()

In [ ]:
len(connections)

In [ ]:
actors["Antonio Banderas"]

In [ ]:
actors["Kevin Bacon"]

In [ ]:
list(nx.all_shortest_paths(G,"Antonio Banderas","Kevin Bacon"))

In [ ]:
['Antonio Banderas', 'Julianne Moore', 'Kevin Bacon']

In [ ]:
G.edges[('Antonio Banderas', 'Julianne Moore')]

In [ ]:
G.edges[('Julianne Moore', 'Kevin Bacon')]

In [ ]:
#from networkx.readwrite import json_graph
# This generates an almost 1GB JSON file with all the data on the graph
#data = json_graph.adjacency_data(G)

## Limit data to make some things faster

In [ ]:
movies.shape

In [ ]:
movies.columns

In [ ]:
select_movies = movies.sort_values(by="votes",ascending=False).iloc[:500]

In [ ]:
select_movies.shape

In [ ]:
G = nx.Graph()
for i, row in select_movies.iterrows():
    pairs = list(combinations(map(lambda x:x.strip(),row["actors"].split(",")),2))
    G.add_edges_from(pairs)
    for pair in pairs:
        G.edges[pair]["movie"] = G.edges[pair].get("movie",[]) + [row["original_title"]]

In [ ]:
actors = G.nodes()
len(actors)

### Degree Centrality

Degree centrality
Historically first and conceptually simplest is degree centrality, which is defined as the number of links incident upon a node (i.e., the number of ties that a node has) over the total number of edges.

In [ ]:
degree = [{k:v} for k,v in nx.degree_centrality(G).items()]
sorted(degree,key= lambda x: list(x.values())[0], reverse=True)

In [ ]:
[(i,c) for i,c in enumerate(degree) if "Kevin Bacon" in c]

### Closeness Centrality
In a connected graph, the normalized closeness centrality (or closeness) of a node is the average length of the shortest path between the node and all other nodes in the graph. Thus the more central a node is, the closer it is to all other nodes.

$$C(x)= \frac{1}{\sum_y d(y,x)}$$

In [ ]:
closeness = [{k:v} for k,v in nx.closeness_centrality(G).items()]
sorted(closeness,key= lambda x: list(x.values())[0], reverse=True)

In [ ]:
[(i,c) for i,c in enumerate(closeness) if "Kevin Bacon" in c]

### Eigenvector Centrality
Eigenvector centrality (also called eigencentrality) is a measure of the influence of a node in a network. It assigns relative scores to all nodes in the network based on the concept that connections to high-scoring nodes contribute more to the score of the node in question than equal connections to low-scoring nodes. Google's PageRank and the Katz centrality are variants of the eigenvector centrality.

For a given graph $G:=(V,E)$ with $|V|$ number of vertices let $A = (a_{v,t})$ be the [[adjacency matrix]], i.e. $a_{v,t} = 1$ if vertex $v$ is linked to vertex $t$, and $a_{v,t} = 0$ otherwise. The relative centrality score of vertex $v$ can be defined:

$$x_v = \frac{1}{\lambda} \sum_{t \in M(v)}x_t = \frac{1}{\lambda} \sum_{t \in G} a_{v,t}x_t$$

where $M(v)$ is a set of the neighbors of $v$ and $\lambda$ is a constant. With a small rearrangement this can be rewritten in vector notation as the eigenvector equation

$$\mathbf{Ax} = {\lambda}\mathbf{x}$$

In [ ]:
eigenvector = [{k:v} for k,v in nx.eigenvector_centrality(G).items()]
sorted(eigenvector,key= lambda x: list(x.values())[0], reverse=True)

In [ ]:
[(i,c) for i,c in enumerate(eigenvector) if "Kevin Bacon" in c]

### Betweenness centrality

Betweenness centrality quantifies the number of times a node acts as a bridge along the shortest path between two other nodes. 

The betweenness of a vertex <math>v</math> in a graph <math>G:=(V,E)</math> with <math>V</math> vertices is computed as follows:

* For each pair of vertices $(s,t)$, compute the shortest paths between them.
* For each pair of vertices $(s,t)$, determine the fraction of shortest paths that pass through the vertex in question (here, vertex $v$).
* Sum this fraction over all pairs of vertices $(s,t)$.

In [ ]:
# Takes forever...
betweenness = [{k:v} for k,v in nx.betweenness_centrality(G).items()]
sorted(betweenness,key= lambda x: list(x.values())[0], reverse=True)

In [ ]:
[(i,c) for i,c in enumerate(betweenness) if "Kevin Bacon" in c]

In [ ]:
florence = nx.florentine_families_graph()

In [ ]:
nx.draw_networkx(florence)

In [ ]:
nx.betweenness_centrality(florence)